<a href="https://colab.research.google.com/github/saurin33/pyspark_basics/blob/master/demographics_filtered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [0]:
# Start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFIlter").getOrCreate()

In [0]:
#  Read the data from S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/demographics.csv"
spark.sparkContext.addFile(url)

In [5]:
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [48]:
# What occupation had the highest salary?
from pyspark.sql.functions import max, min, mean, avg
df.groupBy('occupation').agg(max('salary')).show(1)

+----------+-----------+
|occupation|max(salary)|
+----------+-----------+
| Scientist|         90|
+----------+-----------+
only showing top 1 row



In [43]:
# What occupation had the lowest salary?
from pyspark.sql.functions import min
df.groupBy('occupation').min('salary').orderBy(['min(salary)']).show(1)


+-------------------+-----------+
|         occupation|min(salary)|
+-------------------+-----------+
|Gaming Club Manager|         65|
+-------------------+-----------+
only showing top 1 row



In [36]:
# What is the mean salary of this dataset?
df.select(mean('salary')).show()

+-----------+
|avg(salary)|
+-----------+
|     77.738|
+-----------+



In [53]:
# What is the max and min of the Salary column?
df.agg(max('salary'), min('salary')).show()

+-----------+-----------+
|max(salary)|min(salary)|
+-----------+-----------+
|         90|         65|
+-----------+-----------+



In [46]:
# Show all of the occupations where salaries were above 80k
df.filter(df['salary']>80).show()

+---+--------------------+---+------------+---------+--------+--------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|          occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+--------------------+---------------+------+-------------+
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|              Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|       Choreographer|       Bachelor|    83|West Virginia|
|  8|Charlesetta Steve...| 30|        1.62|       44|       3|          Millwright|         Master|    87|    Louisiana|
| 11|      Shawnee Harmon| 66|        1.63|       78|       5|   Medical Physicist|            PhD|    90|     Delaware|
| 17|   Bernardina Strong| 34|        1.55|       78|       1|           Scientist|            PhD|    90| South Dakota|
| 21|      Kemberly Yates| 41|  

In [49]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this
df.groupby('academic_degree').agg(avg('height_meter'), avg('age')).show()

+---------------+------------------+------------------+
|academic_degree| avg(height_meter)|          avg(age)|
+---------------+------------------+------------------+
|            PhD|1.7438165680473379| 43.15976331360947|
|         Master|1.7549226006191951|43.139318885448915|
|       Bachelor| 1.757227138643069| 42.51032448377581|
+---------------+------------------+------------------+

